# TODO: robots.txt


Note: Web scraping might be against the terms of service of some websites. Make sure you have permission to scrape a website and always respect robots.txt and other rules that a website may have around web scraping.







pip install gtts
pip install sumy
pip install gensim
pip install beautifulsoup4
pip install requests

In [ ]:
from gtts import gTTS
import IPython.display as ipd

# The text you want to convert to speech
text = "Hello Baran Joon, how are you?"

# Language in which you want to convert
# language = 'en'
language = 'en-us' # American accent

# Passing the text and language to the engine, slow=False makes the speech faster
tts = gTTS(text=text, lang=language, slow=False)

# Saving the converted audio in a mp3 file named 'welcome'
tts.save("welcome.mp3")

# Play the saved file
ipd.Audio("welcome.mp3", autoplay=True)


In [ ]:

# URL of the article you want to summarize
url = "https://www.cnn.com/2023/09/03/europe/madrid-residents-storm-dana-downpour-intl/index.html"




In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from bs4 import BeautifulSoup
import requests
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

def fetch_data_from_url(url):
    # Fetch the content from the URL
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Assuming the article text is inside <p> HTML tags
    article_text = ' '.join([p.text for p in soup.find_all('p')])

    return article_text

def summarize_text(text, sentence_count=5):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentence_count)

    return ' '.join(str(sentence) for sentence in summary)


# Fetch and print the summarized data
article_text = fetch_data_from_url(url)
print("Original Article:")
print(article_text)

summary = summarize_text(article_text)
print("\n\nSummarized Article:")
print(summary)


Original Article:

      Residents in Madrid were instructed by city authorities to stay indoors Sunday with Storm Dana set to lash the Spanish capital. 
   
      The state meteorological agency, AEMET, issued a warning for Madrid, Toledo and Cadiz, where Dana is expected to bring strong rains.
   
      According to AEMET, up to 120 liters per square meter of rain could fall over Madrid in 12 hours. 
   
      Valencia, on the Mediterrenean coast, has seen intense rainfall already, AEMAT said.
   
      Madrid’s mayor José Luis Martínez-Almeida advised residents to stay in their houses, calling what was to come an “exceptional situation.”
   
      “Due to the exceptional and abnormal situation, in which rainfall records will be broken, I ask the people of Madrid to stay at home today,” Martinez-Almeida wrote on X, formerly Twitter.
   
      Madrid’s emergency services sent texts to residents warning them of flood risks and advising them against using vehicles. 
   
      Atletico M

In [ ]:


from bs4 import BeautifulSoup
import requests
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from gtts import gTTS
import IPython.display as ipd

def fetch_data_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    article_text = ' '.join([p.text for p in soup.find_all('p')])
    return article_text


def summarize_text(text, sentence_count=5):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentence_count)
    return [str(sentence) for sentence in summary]


def text_to_speech(text):
    language = 'en'
    tts = gTTS(text=text, lang=language, slow=False)
    tts.save("summary.mp3")
    return ipd.Audio("summary.mp3", autoplay=True)


# Fetch, summarize, and read aloud
article_text = fetch_data_from_url(url)
summary_sentences = summarize_text(article_text)
print("Summarized Article:")
for sentence in summary_sentences:
    print(sentence)

text_to_speech(" ".join(summary_sentences))

Summarized Article:
Residents in Madrid were instructed by city authorities to stay indoors Sunday with Storm Dana set to lash the Spanish capital.
The state meteorological agency, AEMET, issued a warning for Madrid, Toledo and Cadiz, where Dana is expected to bring strong rains.
Valencia, on the Mediterrenean coast, has seen intense rainfall already, AEMAT said.
“Due to the exceptional and abnormal situation, in which rainfall records will be broken, I ask the people of Madrid to stay at home today,” Martinez-Almeida wrote on X, formerly Twitter.
Atletico Madrid wrote on X that Sunday’s LaLiga football match against Sevilla had also been postponed due to the storm.
